In [34]:
import numpy as np
from numpy import linalg as LA
import scipy
import itertools as it

In [35]:
def ek(k,dim):
    ek = np.zeros(dim)
    ek[k]=1
    return ek

In [36]:
def grad_neumeric(f,values,h=0.00001):
    dim = len(values)
    derviative = lambda val,pos: (f(*(val+h*pos))-f(*val))/h
    return np.array([derviative(values,ek(i,dim))  for i in range(dim)])

In [37]:
test_f = lambda x,y,z: x**2+y**2+z**2
test_val = np.array([1,2,3])
grad_approx = grad_neumeric(test_f,test_val)
grad_approx

array([2.00001, 4.00001, 6.00001])

In [38]:
def hassian_neumeric(f,values,h=0.00001):
    der_hessian_at_val = []
    der_hessian = lambda vec, eki,ekj: (f(*(vec+h*ekj+h*eki))-f(*(vec+h*ekj))-f(*(vec+h*eki))+f(*vec))/h**2 
    dim = len(values)
    for i in range(dim):
       for j in range(dim):
           val = (der_hessian(values,ek(i,dim),ek(j,dim)))
           der_hessian_at_val.append(val)
    der_hessian_at_val = np.array(der_hessian_at_val).reshape(dim,dim)
    return der_hessian_at_val

In [39]:
test_hf = lambda x,y: x**2+y**2
test_hval = np.array([1,2])
print(grad_neumeric(test_hf,test_hval))
print(hassian_neumeric(test_hf,test_hval))

[2.00001 4.00001]
[[2.00000017 0.        ]
 [0.         2.00000017]]


In [40]:
def gradient_at_val(f,gradient_f,values):
    grad = []
    if gradient_f is None:
        return np.array(grad_neumeric(f,values))
    for function in gradient_f:
        gradient = function(*values)
        grad.append(gradient)
#     print('gradient--->',grad)
    return np.array(grad)

In [41]:
def hassian_at_val(f,gradient2d_f,values):
    hassian = []
    if gradient2d_f is None:
        return hassian_neumeric(f,values)
    for gradient_f in gradient2d_f:
        grad = gradient_at_val(f,gradient_f,values)
        hassian.append(grad)
    return np.array(hassian)

In [42]:
def gradient_direction(function,gradient_f,gradient2d_f,values,newton_direction=False):
    dk = []
    grad = gradient_at_val(function,gradient_f,values)
    if newton_direction:
        dk = - np.dot(LA.inv(hassian_at_val(function,gradient2d_f,values)), grad)
        print('used newton')
        if - np.dot(grad.T,dk)/(LA.norm(grad)*LA.norm(dk)) < 0.05:
            dk = - grad
            print('used grad')
    else:
        dk = - grad
        print('used neg grad')
    return dk
        

In [43]:
def descent_direction(f,gradient_f,values):
    grad = gradient_at_val(f,gradient_f,values)
    p =  - (LA.norm(grad))**2
    return p

In [44]:
def armijo_step_algorithm(f,gradient_f,dk,values,delta):
    ro = 1
    grad_at_values = gradient_at_val(f,gradient_f,values).T
    while f(*(values+ro*dk)) <= f(*values)+ro*delta*np.dot(grad_at_values,dk):
        ro *= 2
    
    while f(*(values+ro*dk)) > f(*values)+ro*delta*np.dot(grad_at_values,dk):
        ro /= 2
    return ro

In [45]:
# Bazaray shetty function
# f = lambda x,y: (x-2)**4 + (x-2*y)**2
# dfx = lambda x,y: 4*(x-2)**3+2*(x-2*y)
# dfy = lambda x,y: -4*(x-2*y)

# dfx11 = lambda x,y: 12*(x-2)**2+2
# dfx12 = lambda x,y: -4

# dfx21 = lambda x,y: -4
# dfx22 = lambda x,y: 8

In [46]:
# rosenbrook function
f = lambda x,y : 100*(y-x*x)**2 + (1-x)**2
dfx = lambda x,y: -400*x*(y-x*x) + 2*x - 2
dfy = lambda x,y: 200*(y-x*x)

dfx11 = lambda x,y: -400*(y-3*x**2)+2
dfx12 = lambda x,y: -400*x

dfx21 = lambda x,y: -400*x
dfx22 = lambda x,y: 200

In [47]:
print(f'dfx numeric: {grad_neumeric(f,[1,2])} ')
print(f'dfx: {dfx(1,2)}  dfy:{dfy(1,2)}')

dfx numeric: [-399.99798996  200.001     ] 
dfx: -400  dfy:200


In [48]:
print(f'dfx numeric: {hassian_neumeric(f,[1,2])} ')
print(f'dfx11: {dfx11(1,2)}  dfx12:{dfx12(1,2)} dfx21:{dfx21(1,2)} dfx22:{dfx22(1,2)}')

dfx numeric: [[ 402.02365881 -400.00216472]
 [-400.00216472  199.99987444]] 
dfx11: 402  dfx12:-400 dfx21:-400 dfx22:200


In [49]:


negative_grad = gradient_at_val(np.array([dfx,dfy]),np.array([1,2]))
# negative_grad = np.array([-400,-200])
delta = 10e-4
step_size = armijo_step_algorithm(f,np.array([dfx,dfy]),negative_grad,np.array([1,2]),delta)
print(step_size)

NameError: name 'negative_grad' is not defined

In [ ]:
from numpy import linalg as LA
def general_descent(iteration,function,gradient_f,gradient2d_f,initial,use_newton=True):
    xk = initial
    counter = 0
    xk1 = 0
    delta = 10**(-3)
    while counter <=iteration:
        print(LA.norm(gradient_at_val(function,gradient_f,xk)))
        dk = gradient_direction(function,gradient_f,gradient2d_f,xk,newton_direction=use_newton)
        step_size = armijo_step_algorithm(function,gradient_f,dk,xk,delta)
        xk1 = xk + step_size*dk
        xk = xk1
        print('xk1 at iterate: {}--->: {}'.format(counter,xk1))
        print('dk at iterate: {}--->: {}'.format(counter,dk))
        print('step_size at iterate: {}--->: {}'.format(counter,step_size))
        counter += 1
        if LA.norm(gradient_at_val(function,gradient_f,xk))<=0.0001:
            break
    return xk1

In [ ]:
general_descent(10000,f,np.array([dfx,dfy]),np.array([[dfx11,dfx12],[dfx21,dfx22]]),np.array([0,10]),use_newton=True)

In [ ]:
general_descent(20000,f,None,None,np.array([0,10]),use_newton=True)

In [ ]:
import matplotlib.pyplot as plt
x = np.arange(-10,10,1)
y = np.arange(-10,10,1)
fx = f(x,y)

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

def draw_function(x,y,f):
    fig = plt.figure(figsize=(20, 10))
    ax = plt.gca(projection='3d')
    s=0.5
    x,y = np.meshgrid(x,y)
    z= f(x,y)
    ax.plot_surface(x, y, z)

In [ ]:
draw_function(x,y,f)